In [1]:
push!(LOAD_PATH, pwd())
using JoinTreeInference: Node, Potential

In [2]:
type Factor
    nodes::Array{String, 1}
    cardinality::Array{Int, 1}
    stride::Array{Int, 1}
    data::Array{Float64, 1}
end
    
function Factor(nodes::Array{String, 1}, cardinality::Array{Int, 1}, data::Array{Int, 1})
    n = length(nodes)
    stride = Array{Int, 1}(n)
    s = 1
    for i in 1:n
        stride[i] = s
        s *= cardinality[i]
    end
    return Factor(nodes, cardinality, stride, data)
end

Factor

In [3]:
function product(factors::Array{Factor, 1})
        num_factors = length(factors)
        vars = collect(union(f1.nodes, f2.nodes))
        var_ids = Dict(j=>i for (i,j) in enumerate(vars))


        n = length(vars)
        cardinality = Array{Int, 1}(n)
        stride = Array{Int, 1}(n)
        
        idmap = Array{Array{Int, 1}, 1}(num_factors)
        for (i, f) in enumerate(factors)
            nv = length(f.nodes)
            idmap[i] = fill(0, n)
            for (j, v) in enumerate(f.nodes)
                id = var_ids[v]
                cardinality[id] = f.cardinality[id]
                idmap[i][id] = j
            end
        end
        
        s = 1
        for i in 1:n
            stride[i] = s
            s *= cardinality[i]
        end

        assignments = fill(1, n)
        ind = fill(1, num_factors)
        num_rows = reduce(*, cardinality)
        data = fill(1.0, num_rows)
        for i in 1:num_rows
            for (j, f) in enumerate(factors)
                data[i] *= f.data[ind[j]]
            end
            for l in 1:n
                assignments[l] += 1
                if assignments[l] == cardinality[l]
                    assignments[l] = 1
                    for (j, id) in enumerate(ind)
                        id2 = idmap[j][id]
                        if id2 > 0
                            ind[j] -= factors[j].stride[id2] * (cardinality[l]-1)
                        end
                    end
                else
                    for (j, id) in enumerate(ind)
                        id2 = idmap[j][id]
                        if id2 > 0
                            ind[j] += factors[j].stride[id2]
                        end
                    end
                    break
                end
            end
        end
        return Factor(vars, cardinality, stride, data)
end

product (generic function with 1 method)

In [4]:
function marginalize(f::Factor, v::Set{String})
    fn = length(f.nodes)
    n = fn - length(v)
    
    vars = Array{String, 1}(n)
    cardinality = Array{Int, 1}(n)
    idmap = fill(0, fn)

    j = 1
    for (i, n) in enumerate(vf.nodes)
        if !(n in v)
            vars[j] = n
            cardinality[j] = f.cardinality[i]
            idmap[i] = j
            j += 1
        end
    end
    
    stride = Array{Int, 1}(n)
    s = 1
    for i in 1:n
        stride[i] = s
        s *= cardinality[i]
    end

    
    data = fill(0.0, reduce(*, cardinality))
    num_rows = reduce(*, f.cardinality)
    assignments = fill(1, fn)
    ind = 1
    for i in 1:num_rows
        data[ind] += f.data[i]
        for (l, n) in enumerate(vf.nodes)
            assignments[l] += 1
            if assignments[l] == f.cardinality[l]
                assignments[l] = 1
                if ! (n in v)
                    ind -= stride[idmap[l]] * (f.cardinality[l]-1)
                end
            else
                if !(n in v)
                    ind += stride[idmap[l]]
                end
            end
        end
    end
    return Factor(vars, cardinality, stride, data)        
end

marginalize (generic function with 1 method)

In [5]:
function divide(f1::Factor, f2::Factor)
end

divide (generic function with 1 method)